In [1]:
import copy
import itertools

In [2]:
#############   Category Class   ###############

# homsets: (object, object) --> list[morphisms]
# src: morphism names --> object names
# target: morphism names --> object names
# comp: (morphism, morphism) --> morphism
# identityMap: objects --> morphisms

class Category:
    def __init__(self, 
                 homsets:dict,
                 src:dict, 
                 target:dict, 
                 comp:dict, 
                 identityMap:dict, 
                 name:str,
                 debug:bool = False) -> None:
        self.src = src
        self.target = target
        self.comp = comp
        self.identityMap = identityMap
        self.debug = debug
        self.homsets = homsets
        self.name = name

        # if data given as homsets then make the source and target dictionaries
        if self.homsets != {}:
            self.sourceAndTargetMake()

        # make object and morphism lists
        self.objects = set(src.values()).union(set(target.values()))
        self.morphisms = list(src.keys())

        # complete data in src, target, comp for identities
        self.completeIdentities()
        
        # remake the homsets, including empty homsets
        self.homsetMake()

        # verify essential properties
        assert set(identityMap.keys()) == self.objects
        assert self.src.keys() == self.target.keys()
        assert self.composabilityCheck()
        assert self.compositionCheck()
        assert self.associativityCheck()
        
    def __repr__(self) -> str:
        return self.__str__()
    
    def __str__(self) -> str:
        return ('\n').join(["Category: ", 
                            "Homsets: " + str(self.homsets), 
                            "Identity map: " + str(self.identityMap), 
                            "Composition: " + str(self.comp),
                            "Name: " + self.name])
    
    # from homset data, make the src and target dictionaries
    def sourceAndTargetMake(self) -> None:
        for (src,target), morphismList in self.homsets.items():
            for thisMorphism in morphismList:
                self.src[thisMorphism] = src
                self.target[thisMorphism] = target


    # given identityMap, fill out missing data in src, target, comp
    def completeIdentities(self) -> None:
        for thisObject in self.objects:
            thisMorphism = self.identityMap[thisObject]
            self.src[thisMorphism] = thisObject
            self.target[thisMorphism] = thisObject
            if thisMorphism not in self.morphisms:
                self.morphisms = self.morphisms + [thisMorphism]
            #self.identityMap[thisObject] = thisMorphism
            for i in self.morphisms:
                if self.src[i] == thisObject:
                    self.comp[(i, thisMorphism)] = i
                if self.target[i] == thisObject:
                    self.comp[(thisMorphism, i)] = i

    # Check that all the composables have matching targets and sources.
    def composabilityCheck(self)-> bool:
        for second,first in self.comp.keys():
            if self.target[first] != self.src[second]:
                if self.debug:
                    print("Composability violation...")
                    print((second,first))
                return False
        return True
    
    # Check that any two composable morphisms have a name
    def compositionCheck(self) -> bool:
        for first in self.morphisms:
            for second in self.morphisms:
                if self.src[second] == self.target[first]:
                    if (second, first) not in self.comp.keys():
                        if self.debug:
                            print("Composables lack a name: ")
                            print("second, first: %s %s" % (second,first))
                        return False
        return True
    
    # Check associativity
    def associativityCheck(self) -> bool:
        for first in self.morphisms:
            for second in self.morphisms:
                for third in self.morphisms:
                    if (self.src[third] == self.target[second]) and (self.src[second] == self.target[first]):
                        if self.comp[(third, self.comp[second, first])] != self.comp[(self.comp[third, second], first)]:
                            if self.debug:
                                print("Associativity failure...")
                                print((third, second, first))
                            return False
        return True

    # from src and target data, make homset dictionary
    def homsetMake(self) -> None:

        for o1 in self.objects:
            for o2 in self.objects:
                self.homsets[(o1, o2)] = []

        for thisMorphism in self.morphisms:
            src = self.src[thisMorphism]
            target = self.target[thisMorphism]
            self.homsets[(src, target)].append(thisMorphism)



#############   Functor Class   ###################

# srcCat: Category
# targetCat: Category
# objectMap: objects --> objects
# morphismMap: morphisms --> morphisms

class Functor:
    def __init__(self, 
                 srcCat: Category, 
                 targetCat: Category, 
                 objectMap: dict, 
                 morphismMap: dict, 
                 debug: bool = False) -> None:
        self.srcCat = srcCat
        self.targetCat = targetCat
        self.objectMap = objectMap
        self.morphismMap = morphismMap
        self.debug = debug

        assert self.identityCheck()
        assert self.homomorphismFunctorCheck()

    def __repr__(self) -> str:
        return self.__str__()
    
    def __str__(self) -> str:
        return ("\n\n".join(["Functor: ", 
                           "source name: " + self.srcCat.name, 
                           "target name: " + self.targetCat.name, 
                           "object map: " + str(self.objectMap), 
                           "morphism map: " + str(self.morphismMap)]))
    
    # verify identity morphisms are mapped correctly
    def identityCheck(self) -> bool:
        for ob,thisID in self.srcCat.identityMap.items():
            if self.morphismMap[thisID] != self.targetCat.identityMap[self.objectMap[ob]]:
                if self.debug:
                    print("Functor does not preserve identities...")
                return False
        return True

    # verify composition is preserved
    def homomorphismFunctorCheck(self) -> bool:
        for first in self.srcCat.morphisms:

            # check that objects and morphisms are mapped consistently
            srcViolation = self.objectMap[self.srcCat.src[first]] != self.targetCat.src[self.morphismMap[first]]
            targetViolation = self.objectMap[self.srcCat.target[first]] != self.targetCat.target[self.morphismMap[first]]
            if srcViolation or targetViolation:
                if self.debug:
                    print("Functor violation...")
                return False
            
            # check that composition is preserved
            for second in self.srcCat.morphisms:
                if self.srcCat.src[second] == self.srcCat.target[first]:
                    srcComp = self.srcCat.comp[(second,first)]
                    targetComp = self.targetCat.comp[(self.morphismMap[second], self.morphismMap[first])]
                    if self.morphismMap[srcComp] != targetComp:
                        if self.debug:
                            print("Functor composition violation...")
                        return False
        return True


##############  Strict 2-Category Class   ##################

# zeroCells: list
# hom: (zeroCell, zeroCell) --> Category
# identityMap: zeroCell --> oneCell (in the category hom(A,A))
# comp: (A: zeroCell, B: zeroCell, C: zeroCell) ---> functor ( Hom(B,C)) x Hom(A, B) --> Hom(A, C) )

class StrictTwoCategory:
    def __init__(self, 
                 zeroCells:list, 
                 hom:dict, 
                 identityMap:dict, 
                 comp:dict, 
                 debug:bool=False) -> None:
        self.zeroCells = zeroCells
        self.hom = hom
        self.identityMap = identityMap
        self.comp = comp
        self.debug = debug

        # verify essential properties
        assert self.identityOneCell()
        assert self.identityTwoCell()
        assert self.associativityOneCell()
        assert self.associativityTwoCell()
        
    def __repr__(self) -> str:
        return self.__str__()
    
    def __str__(self) -> str:
        return ("\n\n").join(["2-category: ", 
                            "zeroCells: " + str(self.zeroCells),
                            "hom: " + str({key:value.name for key,value in self.hom.items()}),
                            "identity map: " + '\n' + str(self.identityMap),
                            "composition: " + '\n' + self.displayComp()])
    
    def displayComp(self) -> str:
        return ('\n\n').join([str(key) + '\n' + str(value) for key,value in self.comp.items()])


    # check that identities work for one cells
    def identityOneCell(self) -> bool:
        for thisZeroCell in self.zeroCells:
            thisIdentityOneCell = self.identityMap[thisZeroCell]
            for zeroCell in self.zeroCells:

                # identity morphism is second
                if (zeroCell,thisZeroCell) in self.hom.keys():
                    thisCompositionFunctor = self.comp[(zeroCell, thisZeroCell, thisZeroCell)]
                    thisOneCellMap = thisCompositionFunctor.objectMap
                    for thisOneCell in self.hom[(zeroCell, thisZeroCell)].objects:
                        if thisOneCellMap[(thisIdentityOneCell, thisOneCell)] != thisOneCell:
                            if self.debug:
                                print("Identity violation for one cells...")
                            return False
                        
                # identity morphism is first
                if (thisZeroCell, zeroCell) in self.hom.keys():
                    thisCompositionFunctor = self.comp[(thisZeroCell, thisZeroCell, zeroCell)]
                    thisOneCellMap = thisCompositionFunctor.objectMap
                    for thisOneCell in self.hom[(thisZeroCell, zeroCell)].objects:
                        if thisOneCellMap[(thisOneCell, thisIdentityOneCell)] != thisOneCell:
                            if self.debug:
                                print("Identity violation for one cells...")
                            return False
                        
        return True
    
    # check that identities work for two cells
    def identityTwoCell(self) -> bool:
        for thisZeroCell in self.zeroCells:
            thisCategory = self.hom[(thisZeroCell, thisZeroCell)]
            thisIdentityOneCell = self.identityMap[thisZeroCell]
            thisIdentityTwoCell = thisCategory.identityMap[thisIdentityOneCell]

            for zeroCell in self.zeroCells:

                # identity morphism is second
                if (zeroCell,thisZeroCell) in self.hom.keys():
                    thisCompositionFunctor = self.comp[(zeroCell, thisZeroCell, thisZeroCell)]
                    thisTwoCellMap = thisCompositionFunctor.morphismMap
                    for thisTwoCell in self.hom[(zeroCell, thisZeroCell)].morphisms:
                        if thisTwoCellMap[(thisIdentityTwoCell, thisTwoCell)] != thisTwoCell:
                            if self.debug:
                                print("Identity violation for two cells...")
                            return False
                        
                # identity morphism is first
                if (thisZeroCell, zeroCell) in self.hom.keys():
                    thisCompositionFunctor = self.comp[(thisZeroCell, thisZeroCell, zeroCell)]
                    thisTwoCellMap = thisCompositionFunctor.morphismMap
                    for thisTwoCell in self.hom[(thisZeroCell, zeroCell)].morphisms:
                        if thisTwoCellMap[(thisTwoCell, thisIdentityTwoCell)] != thisTwoCell:
                            if self.debug:
                                print("Identity violation for two cells...")
                            return False
        
        return True
    
    # verify one cell composition is associative
    def associativityOneCell(self) -> bool:
        for aZeroCell in self.zeroCells:
            for bZeroCell in self.zeroCells:
                for cZeroCell in self.zeroCells:
                    for dZeroCell in self.zeroCells:
                        if (aZeroCell, bZeroCell) in self.hom.keys() and (bZeroCell, cZeroCell) in self.hom.keys() and (cZeroCell, dZeroCell) in self.hom.keys():
                            for abOneCell in self.hom[(aZeroCell, bZeroCell)].objects:
                                for bcOneCell in self.hom[(bZeroCell, cZeroCell)].objects:
                                    for cdOneCell in self.hom[(cZeroCell, dZeroCell)].objects:
                                        abcCompositionFunctor = self.comp[(aZeroCell, bZeroCell, cZeroCell)]
                                        bcdCompositionFunctor = self.comp[(bZeroCell, cZeroCell, dZeroCell)]
                                        abcOneCellMap = abcCompositionFunctor.objectMap
                                        bcdOneCellMap = bcdCompositionFunctor.objectMap
                                        abcFirst = bcdOneCellMap[(cdOneCell, abcOneCellMap[(bcOneCell, abOneCell)])]
                                        bcdFirst = abcOneCellMap[(bcdOneCellMap[(cdOneCell, bcOneCell)], abOneCell)]
                                        if abcFirst != bcdFirst:
                                            if self.debug:
                                                print("Associative violation for one cells...")
                                            return False
        return True
    
    # verify two cell composition is associative
    def associativityTwoCell(self) -> bool:
        for aZeroCell in self.zeroCells:
            for bZeroCell in self.zeroCells:
                for cZeroCell in self.zeroCells:
                    for dZeroCell in self.zeroCells:
                        if (aZeroCell, bZeroCell) in self.hom.keys() and (bZeroCell, cZeroCell) in self.hom.keys() and (cZeroCell, dZeroCell) in self.hom.keys():
                            for abTwoCell in self.hom[(aZeroCell, bZeroCell)].morphisms:
                                for bcTwoCell in self.hom[(bZeroCell, cZeroCell)].morphisms:
                                    for cdTwoCell in self.hom[(cZeroCell, dZeroCell)].morphisms:
                                        abcCompositionFunctor = self.comp[(aZeroCell, bZeroCell, cZeroCell)]
                                        bcdCompositionFunctor = self.comp[(bZeroCell, cZeroCell, dZeroCell)]
                                        abcTwoCellMap = abcCompositionFunctor.morphismMap
                                        bcdTwoCellMap = bcdCompositionFunctor.morphismMap
                                        abcFirst = bcdTwoCellMap[(cdTwoCell, abcTwoCellMap[(bcTwoCell, abTwoCell)])]
                                        bcdFirst = abcTwoCellMap[(bcdTwoCellMap[(cdTwoCell, bcTwoCell)], abTwoCell)]
                                        if abcFirst != bcdFirst:
                                            if self.debug:
                                                print("Associative violation for two cells...")
                                            return False
        return True


###############   Product Category Functions   ##############
    
# construct c x d 
def productCategory(c:Category, d:Category) -> Category:
    src = srcProduct(c, d)
    target = targetProduct(c, d)
    comp = compProduct(c,d)
    identityMap = identityMapProduct(c,d)
    productCat = Category({}, src, target, comp, identityMap, c.name + '_x_' + d.name)
    return productCat

# construct the source dictionary for the product
def srcProduct(c:Category, d:Category) -> dict:
    src = {(m1,m2):(o1, o2) for m1, o1 in c.src.items() for m2,o2 in d.src.items()}
    return src

# construct the target dictionary for the product
def targetProduct(c:Category, d:Category) -> dict:
    target = {(m1,m2):(o1, o2) for m1, o1 in c.target.items() for m2,o2 in d.target.items()}
    return target
    
# construct the composition dictionary for the product    
def compProduct(c:Category, d:Category) -> dict:
    comp = {((m1Source[0],m2Source[0]), (m1Source[1],m2Source[1])):(m1Target, m2Target) for m1Source, m1Target in c.comp.items() for m2Source, m2Target in d.comp.items()}
    return comp

# construct the identity dictionary for the product
def identityMapProduct (c:Category, d:Category) -> dict:
    identityMap = {(ob1, ob2): (id1, id2) for ob1,id1 in c.identityMap.items() for ob2,id2 in d.identityMap.items()}
    return identityMap


##############  ENUMERATION OF ALL FUNCTORS BETWEEN TWO CATEGORIES  ###########


# WARNING: USUALLY INEFFICIENT!  functor enumeration by brute force over ALL maps from morphisms to morphisms
def enumerateFunctorsByMorphism(c:Category, d:Category, numToFind:int=0) -> list:
    functorList = []
    allFunctions = itertools.product(d.morphisms, repeat=len(c.morphisms))
    for thisFunction in allFunctions:
        morphismMap = dict(zip(c.morphisms, thisFunction))
        objectMap = objectMapFromMorphismMap(c, d, morphismMap)
        try:
            potentialFunctor = Functor(c, d, objectMap, morphismMap)
            functorList.append(potentialFunctor)
            if len(functorList) == numToFind:
                return functorList
        except AssertionError:
            pass
    return functorList

# using the functor morphism map from category c to category d determine the object map
def objectMapFromMorphismMap(c:Category, d:Category, morphismMap:dict) -> dict:
    objectMap = []
    for cMorphism, dMorphism in morphismMap.items():
        srcPair = (c.src[cMorphism], d.src[dMorphism])
        objectMap.append(srcPair)
        targetPair = (c.target[cMorphism], d.target[dMorphism])
        objectMap.append(targetPair)

    objectMap = dict(list(set(objectMap)))
    return objectMap

# functor brute force enumeration by first mapping objects and then mapping morphisms consistently
def enumerateFunctorsByObject(c:Category, d:Category, numToFind: int=0) -> list:
    functorList = []
    allObjectFunctions = itertools.product(d.objects, repeat=len(c.objects))
    for thisFunction in allObjectFunctions:
        objectMap = dict(zip(c.objects, thisFunction))

        # create morphismMap for potential functor
        
        cHomsets = list(c.homsets.items())
        homSetFunctions = []
        for (cSrc, cTarget),cMorphismList in cHomsets:
            dSrc = objectMap[cSrc]
            dTarget = objectMap[cTarget]
            thisDHomset = d.homsets[(dSrc, dTarget)]
            homSetFunctions.append(itertools.product(thisDHomset, repeat=len(cMorphismList)))

        allMorphismFunctions =itertools.product(*homSetFunctions)
        for thisMorphismFunction in allMorphismFunctions:
            
            morphismMap = {}
            # map identities to identities
            for thisIdentity in c.identityMap.values():
                morphismMap[thisIdentity] = d.identityMap[objectMap[c.src[thisIdentity]]]
            
            thisMorphismFunction = list(thisMorphismFunction)
            for _, cMorphismList in cHomsets:
                thisDMorphismList = thisMorphismFunction.pop(0)  
                morphismMap.update(dict(zip(cMorphismList,thisDMorphismList)))
            try:
                potentialFunctor = Functor(c, d, objectMap, morphismMap)
                functorList.append(potentialFunctor)
                if len(functorList) == numToFind:
                    return functorList
            except AssertionError:
                pass

    return functorList


#############   ENUMERATE 2-CATEGORY STRUCTURES   ############


# utility function for prepending strings in dictionary keys and values
def prependDictionary(myDict:dict, preString:str):
    newDict = {}
    for key, value in list(myDict.items()):
        if type(key) == tuple:
            newKey = tuple([preString + i for i in key])
        elif type(key) == str:
            newKey = preString + key
        else:
            print("Prepend warning...")

        if type(value) == tuple:
            newValue = tuple([preString + i for i in value])
        elif type(value) == str:
            newValue = preString + value
        else:
            print("Prepend warning...")
        
        newDict[newKey] = newValue
    return newDict

def relabelCategory(cat:Category, src:str, target:str):
    
    def modifyObject(object, src, target):
        return '1_(' + src + '->' + target + ')_' + object[2:]

    def modifyMorphism(morphism, src, target):
        return '2_(' + src + '->' + target + ')_' + morphism[2:].replace('->','=>')

    def modifyHomsets(homsets, objectMap, morphismMap):
        return {(objectMap[src], objectMap[target]):[morphismMap[thisMorphism] for thisMorphism in morphismList] 
                for (src, target),morphismList in homsets.items()}

    def modifyComp(comp, objectMap, morphismMap):
        return {(morphismMap[morphism1], morphismMap[morphism2]):morphismMap[compMorphism] for (morphism1, morphism2), compMorphism in comp.items()}

    def modifyIdentityMap(identityMap, objectMap, morphismMap):
        return {objectMap[thisObject]:morphismMap[thisIdentity] for thisObject, thisIdentity in identityMap.items()}

    objects = cat.objects
    morphisms = cat.morphisms

    objectMap = {thisObject: modifyObject(thisObject, src, target) for thisObject in objects}
    morphismMap = {thisMorphism: modifyMorphism(thisMorphism, src, target) for thisMorphism in morphisms}

    homsets = modifyHomsets(cat.homsets, objectMap, morphismMap)
    comp = modifyComp(cat.comp, objectMap, morphismMap)
    identityMap = modifyIdentityMap(cat.identityMap, objectMap, morphismMap)
    name = cat.name + '_(' + src + ',' + target + ')'

    relabeledCategory = Category(homsets, {}, {}, comp, identityMap, name)
    return relabeledCategory, objectMap, morphismMap

# enumerate all 2-category structures by enumerating all composition functors
def enumerate2Categories(zeroCells: list, hom:dict, identityMap:dict, relabel:bool, debug:bool=False) -> list:

    if relabel:
        hom = copy.deepcopy(hom)
        identityMap = copy.deepcopy(identityMap)

        for (src,target), thisCategory in hom.items():
            relabeledCategory, objectMap, _ = relabelCategory(thisCategory, src, target)
            hom[(src,target)] = relabeledCategory
            if src == target:
                identityMap[src] = objectMap[identityMap[src]]

    twoCategoryList = []
    functorDict = {}
    for o1 in zeroCells:
        for o2 in zeroCells:
            for o3 in zeroCells:
                if ((o1, o2) not in hom.keys()) or ((o2, o3) not in hom.keys()):
                    continue
                c1 = hom[(o1, o2)]
                c2 = hom[(o2, o3)]
                c3 = hom[(o1, o3)]
                thisProduct = productCategory(c2, c1)
                functorDict[(o1, o2, o3)] = enumerateFunctorsByObject(thisProduct, c3, 0)

    functorKeyValueList = functorDict.items()
    functorList = [value for (key, value) in functorKeyValueList]
    objectTupleList = [key for (key, value) in functorKeyValueList]
    allComposition = itertools.product(*functorList)
    for thisComposition in allComposition:
        comp = {}
        try:
            for i,thisPair in enumerate(objectTupleList):
                comp[thisPair] = thisComposition[i]

            potential2Category = StrictTwoCategory(zeroCells, hom, identityMap, comp, debug)
            twoCategoryList.append(potential2Category)
        except:
            pass
    return twoCategoryList

In [30]:
z2_1_1

Category: 
Homsets: {('1_(1->1)_0', '1_(1->1)_0'): ['2_(1->1)_(0=>0)_id', '2_(1->1)_(0=>0)_0']}
Identity map: {'1_(1->1)_0': '2_(1->1)_(0=>0)_id'}
Composition: {('2_(1->1)_(0=>0)_0', '2_(1->1)_(0=>0)_0'): '2_(1->1)_(0=>0)_id', ('2_(1->1)_(0=>0)_id', '2_(1->1)_(0=>0)_id'): '2_(1->1)_(0=>0)_id', ('2_(1->1)_(0=>0)_0', '2_(1->1)_(0=>0)_id'): '2_(1->1)_(0=>0)_0', ('2_(1->1)_(0=>0)_id', '2_(1->1)_(0=>0)_0'): '2_(1->1)_(0=>0)_0'}
Name: z2_(1,1)

In [27]:
z2_0_0,_,_ = relabelCategory(z2, '0', '0')
z2_0_1,_,_ = relabelCategory(z2, '0', '1')
z2_1_1,_,_ = relabelCategory(z2, '1', '1')

objectMap = {('1_(0->0)_0', '1_(0->0)_0'): '1_(0->0)_0'}
F000morphismMap = {('2_(0->0)_(0=>0)_0', '2_(0->0)_(0=>0)_0'): '2_(0->0)_(0=>0)_id', 
                   ('2_(0->0)_(0=>0)_id', '2_(0->0)_(0=>0)_id'): '2_(0->0)_(0=>0)_id', 
                   ('2_(0->0)_(0=>0)_0', '2_(0->0)_(0=>0)_id'): '2_(0->0)_(0=>0)_0', 
                   ('2_(0->0)_(0=>0)_id', '2_(0->0)_(0=>0)_0'): '2_(0->0)_(0=>0)_0'}
F000 = Functor(productCategory(z2_0_0, z2_0_0), z2_0_0, objectMap, F000morphismMap)

objectMap = {('1_(1->1)_0', '1_(1->1)_0'): '1_(1->1)_0'}
F000morphismMap = {('2_(1->1)_(0=>0)_0', '2_(1->1)_(0=>0)_0'): '2_(1->1)_(0=>0)_id', 
                   ('2_(1->1)_(0=>0)_id', '2_(1->1)_(0=>0)_id'): '2_(1->1)_(0=>0)_id', 
                   ('2_(1->1)_(0=>0)_0', '2_(1->1)_(0=>0)_id'): '2_(1->1)_(0=>0)_0', 
                   ('2_(1->1)_(0=>0)_id', '2_(1->1)_(0=>0)_0'): '2_(1->1)_(0=>0)_0'}
F111 = Functor(productCategory(z2_1_1, z2_1_1), z2_1_1, objectMap, F000morphismMap)

objectMap = {('1_(0->1)_0', '1_(0->0)_0'): '1_(0->1)_0'}
F000morphismMap = {('2_(0->1)_(0=>0)_0', '2_(0->0)_(0=>0)_0'): '2_(1->1)_(0=>0)_id', 
                   ('2_(1->1)_(0=>0)_id', '2_(1->1)_(0=>0)_id'): '2_(1->1)_(0=>0)_id', 
                   ('2_(1->1)_(0=>0)_0', '2_(1->1)_(0=>0)_id'): '2_(1->1)_(0=>0)_0', 
                   ('2_(1->1)_(0=>0)_id', '2_(1->1)_(0=>0)_0'): '2_(1->1)_(0=>0)_0'}
F001 = Functor(productCategory(z2_0_1, z2_0_0), z2_0_1, objectMap, F000morphismMap)





zeroCells = ["0_0","0_1"]
hom = {("0_0","0_0"):z2_0_0, ("0_1","0_1"):z2_1_1, ("0_0", "0_1"):z2_0_1}
identityMap = {"0_0":"1_(0->0)_0", "0_1":"1_(1->1)_0"}
#comp = {('0_0', '0_0', '0_0'): F000, ('0_0', '0_0', '0_1'):F001, ('0_0', '0_1', '0_1'):F011, ('0_1', '0_1', '0_1'):F111 }
#twoCat = StrictTwoCategory(zeroCells, hom, identityMap, comp, debug=True)

In [15]:
# Notation: 0_1 is the 0-cell with index 1
zeroCells = ["0_0","0_1"]
hom = {("0_0","0_0"):z2, ("0_1","0_1"):z2, ("0_0", "0_1"):z2}
identityMap = {"0_0":"0_0", "0_1":"0_0"}

# enumerate all 2-category structures
twoCategoryList = enumerate2Categories(zeroCells, hom, identityMap, False, debug=False)
twoCategoryList

[2-category: 
 
 zeroCells: ['0_0', '0_1']
 
 hom: {('0_0', '0_0'): 'z2', ('0_1', '0_1'): 'z2', ('0_0', '0_1'): 'z2'}
 
 identity map: 
 {'0_0': '0_0', '0_1': '0_0'}
 
 composition: 
 ('0_0', '0_0', '0_0')
 Functor: 
 
 source name: z2_x_z2
 
 target name: z2
 
 object map: {('0_0', '0_0'): '0_0'}
 
 morphism map: {('1_(0->0)_id', '1_(0->0)_id'): '1_(0->0)_id', ('1_(0->0)_id', '1_(0->0)_0'): '1_(0->0)_0', ('1_(0->0)_0', '1_(0->0)_id'): '1_(0->0)_0', ('1_(0->0)_0', '1_(0->0)_0'): '1_(0->0)_id'}
 
 ('0_0', '0_0', '0_1')
 Functor: 
 
 source name: z2_x_z2
 
 target name: z2
 
 object map: {('0_0', '0_0'): '0_0'}
 
 morphism map: {('1_(0->0)_id', '1_(0->0)_id'): '1_(0->0)_id', ('1_(0->0)_id', '1_(0->0)_0'): '1_(0->0)_0', ('1_(0->0)_0', '1_(0->0)_id'): '1_(0->0)_0', ('1_(0->0)_0', '1_(0->0)_0'): '1_(0->0)_id'}
 
 ('0_0', '0_1', '0_1')
 Functor: 
 
 source name: z2_x_z2
 
 target name: z2
 
 object map: {('0_0', '0_0'): '0_0'}
 
 morphism map: {('1_(0->0)_id', '1_(0->0)_id'): '1_(0->0)_id', 

### Examples: defining categories, the product, and a functor

In [4]:
# define two identical categories and their product
homsets = {('0_0','0_0'): ['1_(0->0)_id'],
           ('0_1','0_1'): ['1_(1->1)_id'],
           ('0_2','0_2'): ['1_(2->2)_id'],
           ('0_3','0_3'): ['1_(3->3)_id'],
           ('0_0','0_1'): ['1_(0->1)_0'],
           ('0_1','0_2'): ['1_(1->2)_0'],
           ('0_2','0_3'): ['1_(2->3)_0'],
           ('0_0','0_2'): ['1_(0->2)_0'],
           ('0_0','0_3'): ['1_(0->3)_0'],
           ('0_1','0_3'): ['1_(1->3)_0']}

comp = {('1_(1->2)_0','1_(0->1)_0'): '1_(0->2)_0',
        ('1_(2->3)_0','1_(1->2)_0'): '1_(1->3)_0',
        ('1_(2->3)_0','1_(0->2)_0'): '1_(0->3)_0',
        ('1_(1->3)_0','1_(0->1)_0'): '1_(0->3)_0'}

identityMap = {'0_0': '1_(0->0)_id',
               '0_1': '1_(1->1)_id',
               '0_2': '1_(2->2)_id',
               '0_3': '1_(3->3)_id'}

c1 = Category(homsets, {}, {}, comp, identityMap, '4_cat')
c2 = Category(homsets, {}, {}, comp, identityMap, '4_cat')

productC1C2 = productCategory(c1, c2)

# define the identity functor
objectMap = {i:i for i in c1.objects}
morphismMap = {i:i for i in c1.morphisms}
F = Functor(c1, c2, objectMap, morphismMap)


# define another category
homsets = {('0_0','0_0'): ['1_(0->0)_id', '1_(0->0)_0'],
           ('0_1','0_1'): ['1_(1->1)_id', '1_(1->1)_0'],
           ('0_0','0_1'): ['1_(0->1)_0', '1_(0->1)_1']}

comp = {('1_(0->0)_0', '1_(0->0)_0'): '1_(0->0)_id',
        ('1_(1->1)_0', '1_(1->1)_0'): '1_(1->1)_id',
        ('1_(0->1)_0', '1_(0->0)_0'): '1_(0->1)_0',
        ('1_(0->1)_1', '1_(0->0)_0'): '1_(0->1)_1',
        ('1_(1->1)_0', '1_(0->1)_0'): '1_(0->1)_0',
        ('1_(1->1)_0', '1_(0->1)_1'): '1_(0->1)_1'}
identityMap = {'0_0': '1_(0->0)_id', '0_1': '1_(1->1)_id'}

c3 = Category(homsets, {}, {}, comp, identityMap, '2_2')


# Define the group z2 as a category
homsets = {('0_0', '0_0'): ['1_(0->0)_id', '1_(0->0)_0']}
comp = {('1_(0->0)_0', '1_(0->0)_0'): '1_(0->0)_id'}
identityMap = {'0_0': '1_(0->0)_id'}
z2 = Category(homsets, {}, {}, comp, identityMap, 'z2')

### To define a strict 2-category:

    1. specify 0-cells
    2. specify categories Hom(A,B) for all 0-cells A,B
    3. specify identity 1-cell in Hom(A,A) for each 0-cell A (identity 2-cell follows automatically)
    4. specify composition functor Hom(B,C) x Hom(A,B) --> Hom(A,C) respecting identities and associativity

In [14]:
# Notation: 0_1 is the 0-cell with index 1
zeroCells = ["0_0","0_1"]
hom = {("0_0","0_0"):z2, ("0_1","0_1"):z2, ("0_0", "0_1"):z2}
identityMap = {"0_0":"0_0", "0_1":"0_0"}

# enumerate all 2-category structures
twoCategoryList = enumerate2Categories(zeroCells, hom, identityMap, True)
print(twoCategoryList)

[]


### Example: Enumerate all functors between two categories

In [12]:
functorList = enumerateFunctorsByObject(productCategory(z2,z2), z2, 0)
print(functorList)

# WARNING: INEFFICIENT
#functorList = enumerateFunctorsByMorphism(z2, z2, 0)

[Functor: 

source name: z2_x_z2

target name: z2

object map: {('0_0', '0_0'): '0_0'}

morphism map: {('1_(0->0)_id', '1_(0->0)_id'): '1_(0->0)_id', ('1_(0->0)_id', '1_(0->0)_0'): '1_(0->0)_id', ('1_(0->0)_0', '1_(0->0)_id'): '1_(0->0)_id', ('1_(0->0)_0', '1_(0->0)_0'): '1_(0->0)_id'}, Functor: 

source name: z2_x_z2

target name: z2

object map: {('0_0', '0_0'): '0_0'}

morphism map: {('1_(0->0)_id', '1_(0->0)_id'): '1_(0->0)_id', ('1_(0->0)_id', '1_(0->0)_0'): '1_(0->0)_id', ('1_(0->0)_0', '1_(0->0)_id'): '1_(0->0)_0', ('1_(0->0)_0', '1_(0->0)_0'): '1_(0->0)_0'}, Functor: 

source name: z2_x_z2

target name: z2

object map: {('0_0', '0_0'): '0_0'}

morphism map: {('1_(0->0)_id', '1_(0->0)_id'): '1_(0->0)_id', ('1_(0->0)_id', '1_(0->0)_0'): '1_(0->0)_0', ('1_(0->0)_0', '1_(0->0)_id'): '1_(0->0)_id', ('1_(0->0)_0', '1_(0->0)_0'): '1_(0->0)_0'}, Functor: 

source name: z2_x_z2

target name: z2

object map: {('0_0', '0_0'): '0_0'}

morphism map: {('1_(0->0)_id', '1_(0->0)_id'): '1_(0->0

In [13]:
len(functorList)

4

## Scratch